# Пример применения библиотеки

Пример применения бибилиотеки на kaggle данных Титаника

## Задача

Применить библиотеку AutoML на данных  kaggle задачи Титаника. </br>
Основная задача с помощью бибилотеки: </br>
1. Проанализировать данные и подсказать о возможных проблемах </br>
    * 1.1 Отчет по таргету датафрема
    * 1.2 Отчет по фичам датафрема
2. Оптимизировать память датафрейма для дальнешего обучения </br>
3. Перебрать различные pipeline(ы), те автоматически: </br>
    * 3.1 Заполнить пропуски в данных </br>
        * 3.1.1 Заполнить числовых пропусков </br>
        * 3.1.2 Заполнить категориальных пропусков </br>
    * 3.2 Преборазование признаков </br>
        * 3.2.1 Преобразование числовых признаков </br>
        * 3.2.1 Преобразование категориальных признаков
    * 3.3 Сделать перербор моделей, их гиперпараметров и сформировать отчет </br>
4. Сформированые отчеты
5. Что можно улучшить

Опишем каждый из пунктов выше подробнее, применив бибилотеку. <br>
Не всеописанные автотесты еще реализованы, но их реализация не представляется сложной.

In [1]:
#Отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# Подключаем графические модули:
# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# будем отображать графики прямо в jupyter'e
%pylab inline

#графики в svg выглядят более четкими
#%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from matplotlib import pyplot


#Подключаем модули для работы с ML
import numpy as np
import pandas as pd

# Для разбияния изначальных данных
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


# from collections.abc import Iterable
# from itertools import product
# from sklearn.preprocessing import MinMaxScaler

Populating the interactive namespace from numpy and matplotlib


In [9]:
#  Написанные модули
from mytransformers import FeatureSelector, ModifiedSimpleImputer, ModifiedFeatureUnion, MyLEncoder
from AutoMLtransformers import AutoMlClassification

In [ ]:
https://zablo.net/blog/post/pandas-dataframe-in-scikit-learn-feature-union/

# Считываем данные

In [10]:
# Считываем данные и разбиваем их на Train и Test

df = pd.read_csv('../ml_data/train.csv')
target_col = 'Survived'

y = df[target_col]
X = df.drop([target_col], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# 1. Проанализировать данные и подсказать о возможных проблемах

In [ ]:
# Создаем экземпляр класса

#reports_path='/reports'
au1 = AutoMlClassification(df, X_train, X_test, y_train, y_test, target_col)

## 1.1 Отчет по таргету датафрема

Так дана задача бинарной классификации, выведем отчет: <br>
   * по количеству таргета по различным фичам
   * по соотношению полодительнего и отрицатльного класса

In [4]:
# Формируем отчет по таргету

au1.get_report_about_target(df, target_col)

Отчет по статистикам по таргету сформирован


,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,count,count_norm
0,0,549,549,549,549,424,549,549,549,549,68,549,549,0.616162
1,1,342,342,342,342,290,342,342,342,342,136,340,342,0.383838


**Резюме:**  Так сверху формирования этого отчета наложено тестирование:  
        * количество первого класса.
        Если количество переменных будет меньше например 100 единиц, то можно выводить ошибку, недостаточно данных

In [6]:
# Формируем отчет по фичам

au1.get_report_about_features(df)

Отчет по статистикам по фичам сформирован


,features,count,unique,top,freq,mean,std,min,25%,50%,75%,max,feat_count_null,feat_type
0,PassengerId,891,891,NaN,NaN,446,257.354,1,223.5,446,668.5,891,0,int64
1,Survived,891,2,NaN,NaN,0.383838,0.486592,0,0,0,1,1,0,int64
2,Pclass,891,3,NaN,NaN,2.30864,0.836071,1,2,3,3,3,0,int64
3,Name,891,891,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
4,Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
5,Age,714,89,NaN,NaN,29.6991,14.5265,0.42,20.125,28,38,80,177,float64
6,SibSp,891,7,NaN,NaN,0.523008,1.10274,0,0,0,1,8,0,int64
7,Parch,891,7,NaN,NaN,0.381594,0.806057,0,0,0,0,6,0,int64
8,Ticket,891,681,CA. 2343,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
9,Fare,891,248,NaN,NaN,32.2042,49.6934,0,7.9104,14.4542,31,512.329,0,float64


**Резюме:**  По этому отчету можно понять и наложить поверх тесты <br>
   * **features** - какие столбца присутсвуют в данных
   * **count** - количество переменных по каждой из фичей
   * **unique** - количество уникальных переменных по каждой из фичей
   * **feat_count_null** - количество пропусков: NaN переменных
   * **top** - название наиболее употребляемого значения
   * **freq** - частота наиболее употребляемого значения
   * **числовые характеристики** 

**Автоматические тесты**:
   * По числовым харакетристикам, если std = 0 или что тоже самое min значение = max значению, можно выводить сообщение.<br> 
   Что эту фичу можно выкинуть из обучения, тк ее значение констатна во всем столбцы и для единичек, и для ноликов, тем самым она не несет никакого смысла.

# 2. Оптимизировать память датафрейма для дальнешего обучения

В будущем возникнет такая проблема, если в AirFlow будет крутиться много однотипных моделей. <br>
Когда многие из них переведем в pandas для обучения, то возникнет проблема с памятью на локальной машине. <br>
Поэтому память нужно при любых возможностях оптимизировать. <br> 
Решить проблемы с памятью можно конечно, переписав обучение полностью на spark, тогда награзука распределиться на весь кластер,
но оптимизацией памяти все равно не стоит пренебрегать.

Например: <br>
Если колонка таргета состоят только из 0 и 1, не имеет смысла отводить под нее int64, тк

* int8 - 8 бит
* int64 - 64 бита <br>

разница составляет в 8 раз, если умножить это на количство столбцов и предположим на 1млн примеров, разница в памяти будет существенной


In [7]:
# Проверим какие колонки можно оптимизировать по памяти

au1.optimize_types(df)

Col name : PassengerId Col min_value : 1 Col max_value : 891 Optimized Class : int16
Col name : Survived Col min_value : 0 Col max_value : 1 Optimized Class : int8
Col name : Pclass Col min_value : 1 Col max_value : 3 Optimized Class : int8
Col name : SibSp Col min_value : 0 Col max_value : 8 Optimized Class : int8
Col name : Parch Col min_value : 0 Col max_value : 6 Optimized Class : int8


**Вывод** Колонки ниже можно оптимизировать по памяти, приведях их к соотвествующим значениям. <br>
Если в функции указать значение inplace=True, то приведение произойдет автоматически

# 3. Перебрать различные pipeline(ы), те автоматически:

Реализована функциональность перебора всех этапов модели для достижения наилучшего результат. <br>
Построение модели происходит при помощи следующего pipeline.

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git

Pipeline([
    # Use FeatureUnion to combine the features
    ('union', ModifiedFeatureUnion(
        transformer_list=[
             # categorical features
            ('categorical', Pipeline([
                 ('selector', FeatureSelector(columns = cat_features)),
                 ('imputer_categorical', imputer_categorical(**current_imputer_categorical_params)),
                 ('label_encoding', encoder(**current_encoder_param))
            ])),
            # numeric features
            ('numeric', Pipeline([
                 ('selector', FeatureSelector(columns = digits_features)),
                 ('imputer_numeric', imputer_numeric(**current_imputer_numeric_params)),
                 ('scaler', scaler(**current_scaler_param))
            ])),
        ])),
    # Use model fit
    ('model', clf(**tmp_params)),
])


Pipeline включает в себя работу как числовыми, так и категориальными призакнами: 
     * 1) Работа с категриальными признаками 'categorical': 
        *  selector - выбираем, интересующие категориальные колонки.
        *  imputer_categorical - заполняем пропуски в категориальных признаках
        *  label_encoding - кодируем категориальные признаки <br>
        
     * 2) Работа с категриальными признаками 'numeric':
        *  selector - выбираем, интересующие числовые колонки.
        *  imputer_numeric - заполняем пропуски в числовых признаках
        *  scaler - нормализуем данные, тк некоторым моделям нужно подавать нормализованные данные <br>
        
     * 3) Применение модели машинного обучения
        *  model - применяем модель машинного обучения
    
С помощью такого pipeline осуществляется перебор различных моделей, а также их различных гипер-параметров. <br>

**Улучшение:** Добавить преобразование временных признаков, а также написать различные классы для заполнения пропусков в данных и <br> 
преобразование числовых и категориальных признаков.

## 3.1 Заполнить пропуски в данных

Был модифицирован класс SimpleImputer для более лучшей работы

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git

class ModifiedSimpleImputer(SimpleImputer):
    
    def transform(self, X):
        return pd.DataFrame(super().transform(X))

### 3.1.1 Заполнить числовых пропусков


Заполение приосходит при помощи строчки кода
**imputer_numeric( ** current_imputer_numeric_params)**, <br> 
где происходит перебор моделей по заполнению пропусков, а также их гипер-параметров.

На данный момент для заполнения пропощенных значений используется только один способ ModifiedSimpleImputer, и перебор <br>
по его гипер-параметрами: заполнение с помощью 0 и -1. <br> 
Но ни что не мешает добавить еще несколько способов, а также их гипер-параметров, по которым осуществить перебор.

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git

imputer_numeric_list = [ModifiedSimpleImputer]
imputer_numeric_name = ['ModifiedSimpleImputer'] 

simpleimputer_params_numeric = {'fill_value': [0, -1],
                                'strategy': ['constant']}
params_numeric_list = [simpleimputer_params_numeric]        


### 3.1.2 Заполнить категориальных пропусков

Заполение приосходит при помощи строчки кода
**imputer_categorical( ** current_imputer_categorical_params)**, <br> 
где происходит перебор моделей заполения, а также их гипер-параметров.

На данный момент для заполнения пропощенных значений используется только один способ ModifiedSimpleImputer, и  <br>
гипер-параметрами ниже, но ни что не мешает добавить еще несколько способов, а также гипер-параметры, <br>
по которым осуществляется перерборд для новых способов.

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git

imputer_cat_list = [ModifiedSimpleImputer]

simpleimputer_params_cat = {'strategy': ['constant'],
                            'fill_value': ['missing']}

params_cat_list = [simpleimputer_params_cat]        

**Улучшения:** Числовые и категориалбны признаки можно заполнять на основе балансов классов в категориальных переменных. <br>
<br>
*Например:* пусть будет известна информация, что М курит 60%,а Ж 40%, тогда колнку курящий или не куриящий, можно заполнить, <br>
на распределении колонка пола, те у всех М и Ж у которых не проставлен признак "курения", заполнить соотвественно с таким же балансом.<br>
Далее, например, можно сделать group by не только по полу, но и по возрасту и заполнить недостающие переменные таким же образом, как описано выше и тд...

## 3.2 Преборазовать признаки

### 3.2.1 Преобразование числовых признаков

Преобразование числвых признаков приосходит при помощи строчки кода
**encoder( ** current_encoder_param))**, <br> 
где происходит перебор моделей, а также их гипер-параметров.

В текущей задачи использовался только один способ нормализации данных с помощью класса из sklearn: MinMaxScaler и <br>
перебор по его гипер-параметра 'feature_range' <br> 

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git

scaler_list = [MinMaxScaler]
scaler_name_list = ['MinMaxScaler'] 

scaler_params = {'feature_range': [(0,1) , (2,3)]}
params_scaler_list = [scaler_params]        


**Улучшения:** Можно добавить другие способы нормализации данных и запустить их перебор, чтобы выбрать лучший. <br>
Функциональность для этого уже вся реализована.

### 3.2.1 Преобразование категориальных признаков

In [ ]:
В текущей задачи использовался только один способ кодирования при помощи модификации класса из sklearn: LabelEncoder

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git

class MyLEncoder():
    
    def transform(self, X, **fit_params):
        enc = LabelEncoder()
        enc_data = []
        for i in list(X.columns):
            X[i] = X[i].astype(str)
            encc = enc.fit(X[i])
            enc_data.append(encc.transform(X[i]))
        return np.asarray(enc_data).T
    
    def fit_transform(self, X,y=None,  **fit_params):
        self.fit(X,y,  **fit_params)
        return self.transform(X)
    def fit(self, X, y, **fit_params):
        return self 

**Улучшения:** Можно добавить другие способы кодирования перменных, например взять из библиотеки: <br>
https://contrib.scikit-learn.org/category_encoders/

##  3.3 Сделать перербор моделей, их гиперпараметров и сформировать отчет

В этом пункте происходит перебор всех элементов pipeline и их гиперпараметров для нахождения наилучшей комбинации на фолдах:<br>
    * Способы заполнения пропусков с числовых и категориальных переменных
    * Нормализация числовых и кодирование категориальных переменных
    * Обучение различных моделей
    
По итогу формируется **отчет** в котором содержатся все комбинации друг с другом этапов pipeline, а также <br>
различиные метрики классификации: roc_auc, recall, precision

In [ ]:
# не запускать, пришлось закомментировать из-за некорректного отображения в git


# Заполнение пропусков категориальных и числовых фичей
imputer_numeric_list = [ModifiedSimpleImputer]
simpleimputer_params_numeric = {'fill_value': [0, -1],
                                'strategy': ['constant']}


imputer_cat_list = [ModifiedSimpleImputer]
simpleimputer_params_cat = {'strategy': ['constant'],
                            'fill_value': ['missing']}


# Нормализация числовых данных и кодирование категориальных фичей
scaler_list = [MinMaxScaler]
scaler_params = {'feature_range': [(0,1) , (2,3)]}

label_encoding = [MyLEncoder]


# Классификаторы
classifiers = [LogisticRegression, RandomForestClassifier] 

logistic_params = {'penalty': ('l1', 'l2'),
                           'C': (.01,5)}

forest_params = {'n_estimators': [10, 30],
                'criterion': ('gini', 'entropy')}



In [5]:
%%time

# Вызов функции перебора параметров pipeline

au1.fit_report(X_train, X_test, y_train, y_test)

imputer_numeric_name fill_na ModifiedSimpleImputer

 Параметры:  {'fill_value': 0, 'strategy': 'constant'}
scaler_name MinMaxScaler

 Параметры scaler:  {'feature_range': (0, 1)} 

classifiers_name LogisticRegression
Параметры classifiers:  {'penalty': 'l1', 'C': 0.01}
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  416 207
Параметры classifiers:  {'penalty': 'l1', 'C': 5}
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  416 207
Параметры classifiers:  {'penalty': 'l2', 'C': 0.01}
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  416 207
Параметры classifiers:  {'penalty': 'l2', 'C': 5}
Размер тренировочного / тестового датасета:  415 208
Размер тренировочного / тестового датасета:  41

In [37]:
# Пример сформированного отчета
report_model = pd.read_csv("../reports/model_report.csv")
report_model.head()

,C,classifier_name,classifier_params,feature_range,fill_value,fit_time,fold,imputer_name,imputer_params,penalty,predict_time,roc_auc,scaler_name,scaler_params,strategy,criterion,n_estimators
0,0.01,LogisticRegression,"penalty=l1, C=0.01","(0, 1)",0.0,0.018213,0.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.013611,0.501537,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
1,0.01,LogisticRegression,"penalty=l1, C=0.01","(0, 1)",0.0,0.016642,1.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.009462,0.500000,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
2,0.01,LogisticRegression,"penalty=l1, C=0.01","(0, 1)",0.0,0.015719,2.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.016188,0.507942,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
3,5.00,LogisticRegression,"penalty=l1, C=5","(0, 1)",0.0,0.016453,0.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.008841,0.788441,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
4,5.00,LogisticRegression,"penalty=l1, C=5","(0, 1)",0.0,0.016344,1.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.008805,0.743779,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN


# 4. Сформированые отчеты

Все полученные выше отчеты сохраняются в папку: report для возможности дальнейшего ручного анализа.

In [27]:
target_report = pd.read_csv("../reports/target_report.csv")
target_report.head()

,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,count,count_norm
0,0,549,549,549,549,424,549,549,549,549,68,549,549,0.616162
1,1,342,342,342,342,290,342,342,342,342,136,340,342,0.383838


In [29]:
features_report = pd.read_csv("../reports/features_report.csv")
features_report.head(15)

,features,count,unique,top,freq,mean,std,min,25%,50%,75%,max,feat_count_null,feat_type
0,PassengerId,891.0,891,NaN,NaN,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000,0,int64
1,Survived,891.0,2,NaN,NaN,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000,0,int64
2,Pclass,891.0,3,NaN,NaN,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000,0,int64
3,Name,891.0,891,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
4,Sex,891.0,2,male,577.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
5,Age,714.0,89,NaN,NaN,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000,177,float64
6,SibSp,891.0,7,NaN,NaN,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000,0,int64
7,Parch,891.0,7,NaN,NaN,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000,0,int64
8,Ticket,891.0,681,CA. 2343,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,object
9,Fare,891.0,248,NaN,NaN,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292,0,float64


In [7]:
report_model = pd.read_csv("../reports/model_report.csv")
report_model.head()

,C,classifier_name,classifier_params,feature_range,fill_value,fit_time,fold,imputer_name,imputer_params,penalty,predict_time,roc_auc,scaler_name,scaler_params,strategy,criterion,n_estimators
0,0.01,LogisticRegression,"penalty=l1, C=0.01","(0, 1)",0.0,0.018213,0.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.013611,0.501537,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
1,0.01,LogisticRegression,"penalty=l1, C=0.01","(0, 1)",0.0,0.016642,1.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.009462,0.500000,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
2,0.01,LogisticRegression,"penalty=l1, C=0.01","(0, 1)",0.0,0.015719,2.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.016188,0.507942,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
3,5.00,LogisticRegression,"penalty=l1, C=5","(0, 1)",0.0,0.016453,0.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.008841,0.788441,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN
4,5.00,LogisticRegression,"penalty=l1, C=5","(0, 1)",0.0,0.016344,1.0,ModifiedSimpleImputer,"fill_value=0, strategy=constant",l1,0.008805,0.743779,MinMaxScaler,"feature_range=(0, 1)",constant,NaN,NaN


# 5. Что можно улучшить

 1. **Распараллеливание:**
 <br>Основная сложность всей задачи состоит в том, что происходит перебор большого числа параметров, если добавить в pipeline этапы:<br>
генерации новых фичей и финальный отбор признаков, то количество перебора возрастет еще во много раз. Тк построение каждого конкретного pipeline c заданными элементами является независимым процессом, то нужно применять распараллеливание.
Это возможно сделать: <br>
  * 1) с помощью spark, переписав все на pyspark, то решится и еще одна проблема с памятью, тк не будется делать toPandas(),
       следовательно тратиться оперативная память конкретной машинки. Нагрузка будет распределяться на всем сервере.
  * 2) запустить некоторые этапы на GPU  <br>

<br>2. **Новая функциональность:**
  * 1) *Поиск ликов*  <br>
      Переодически встречаются лики в данных. Их можно проверить с помощью автоматического построение простых моделей: пни или Logistic regression по каждой из фичей. Алгоритм выглядит следующим образом: из всего dataframe берется только одна фича и таргет и по ней строится модель. Так делается по всем фичам. Если по какой-то из фичей скор у модели слишком большой, либо это killer фича, либо ликовая.  <br>
 <br>
  * 2) *Автоматическая генерация новых признаков.* <br>
    минимум составление concat колонок категориальных признаков(перемножение их между собой)
    взятие различных математических операций из существующих числовых признаков
    
  